# Tutorial 4: Scheduling with CP: Some Basics 

Today we discover a new family of problems, called scheduling. Scheduling problems are widely present in real life applications such as timetabelling, transportation, project management, and manufacturing. We consider a particular problem called the job shop scheduling problem.  In this problem, we are given a set of $n$ jobs: $J_1, J_2, \ldots,  J_n$ and a set of $m$ machines $M_1, M_2, \ldots,  M_m$. 
Each job $J_i$ is defined by a set of $m$ (non-preemptive) tasks $T_{i,1} \ldots T_{i,m}$. Every task $T_{i,k}$ is associated with a duration $p_{i,k}$ and is supposed to be scheduled on machine $M_k$. 

The problem has two sets of constraints: 

 - Precedence constraints: Each job is associated with an order of tasks to respect when scheduling the different tasks.
 - Disjunctive constraints: Each machine can process only one task at a given time


The standard optimisation version of this problem asks to minimize the makespan, i.e., the total scheduling time.


Constraint programming has been widely and successfully used to solve scheduling problems. Many global constraints have been proposed. CP solvers often offer a dedicated library for scheduling. 
Please have a look at the diffrent features proposed in docplex here http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=scheduling#scheduling-functions 


In this tutorial, we will be using (only): 
 - Interval variables for the different variables of the problem:  http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html?highlight=interval_var#docplex.cp.expression.interval_var
 
 - end_before_start constraints to model precedence constraints : http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=end_before_start#docplex.cp.modeler.end_before_start

- no_overlap global constraint : http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=no_overlap#docplex.cp.modeler.no_overlap


The format for a job shop scheduling instance respects the following syntax: 
 - The first line containts only $n$ $m$ in this order  ($n$ is the number of jobs and $m$ is the number of machines)
 - Then $n$ lines are given. Each line $i$ is associated to the job $J_i$ and contains exactly $2 m$ integers $x^i_1$ $d^i_1$ $x^i_2$ $d^i_2$ $\ldots$ $x^i_n$ $d^i_n$. Each $x^i_k$ is the $k^{th}$ machine required by the $k^{th}$ task of the job $J_i$, and $d^i_k$ represents its duration. 

Consider for example the data file instance.data : 
    
 6  6
     
2   1  0   3  1   6  3   7  5   3  4   6

1   8  2   5  4  10  5  10  0  10  3   4

2   5  3   4  5   8  0   9  1   1  4   7
 
 1   5  0   5  2   5  3   3  4   8  5   9
 
 2   9  1   3  4   5  5   4  0   3  3   1
 
 1   3  3   3  5   9  0  10  4   4  2   1


This instance has $6$ jobs and $6$ machines. The first job requires the execution of task $T_{1,2}$ (on machine $2$) of duration $1$, 
then $T_{1,0}$ (on machine $0$) of duration $3$, etc. 

Write a simple python code to parse the data file instance.data

In [1]:
from config import setup
setup()

## install docplex first with $pip install docplex
from docplex.cp.model import *
from docplex.cp.config import get_default
import display
%matplotlib inline

In [2]:
def parse(file):
    with open(file, 'r') as f:
        n, m = [int(x) for x in f.readline().split()]
        jobs=[]
        for line in f:
            job = [int(x) for x in line.split()]
            job = [job[i:i+2] for i in range(0, len(job), 2)]
            jobs.append(job)
        return(n,m,jobs)

In [3]:
n,m,jobs = parse('example.data')
print(n,m)
print(jobs)

6 6
[[[2, 1], [0, 3], [1, 6], [3, 7], [5, 3], [4, 6]], [[1, 8], [2, 5], [4, 10], [5, 10], [0, 10], [3, 4]], [[2, 5], [3, 4], [5, 8], [0, 9], [1, 1], [4, 7]], [[1, 5], [0, 5], [2, 5], [3, 3], [4, 8], [5, 9]], [[2, 9], [1, 3], [4, 5], [5, 4], [0, 3], [3, 1]], [[1, 3], [3, 3], [5, 9], [0, 10], [4, 4], [2, 1]]]


Create a matrix machine that satisfies: machine[i][k] is the machine of the $k^{th}$ task of job $i$ (one line)

In [4]:
machine=[[job[k][0] for k in range(len(job))] for job in jobs]

print(machine)

[[2, 0, 1, 3, 5, 4], [1, 2, 4, 5, 0, 3], [2, 3, 5, 0, 1, 4], [1, 0, 2, 3, 4, 5], [2, 1, 4, 5, 0, 3], [1, 3, 5, 0, 4, 2]]


Create a matrix duration that satisfies: duration[i][k] is the duration of the $k^{th}$ task of job $i$ (one line)

In [5]:
duration=[[job[k][1] for k in range(len(job))] for job in jobs]

print(duration)

[[1, 3, 6, 7, 3, 6], [8, 5, 10, 10, 10, 4], [5, 4, 8, 9, 1, 7], [5, 5, 5, 3, 8, 9], [9, 3, 5, 4, 3, 1], [3, 3, 9, 10, 4, 1]]


Compute a naive upper bound for the makespan. Note: this upper bound will be used as an upper bound for every interval variable we create.

In [6]:
upper_bound = sum(di for d in duration for di in d)

print(upper_bound)

197


Create a CpoModel() and the different interval variables you need to solve this problem (don't forget to use the upper bound you computed earlier)

In [52]:
mdl = CpoModel(name='model')

#durées, on a n*m durées (autant que de tasks), chacune comprise entre 0 et upper_bound
vard=[]
for line in duration:
    vard.append([])
    for d in line:
        vard[-1].append(mdl.interval_var(start=(0, upper_bound), end=(0, upper_bound), length=d, name='d'))

Post the precedence constraints using the end_before_start constraints 

In [53]:
#Precedence constraints: each job is associated with an order of tasks to respect when scheduling the different tasks
for j in range(n):
    lk = len(jobs[j])
    for k in range(lk-1):
        mdl.add(end_before_start(vard[j][k],vard[j][k+1]))

Post the disjunctive constraints using the no-overlap constraints 

In [54]:
#Disjunctive constraints: each machine can process only one task at a given time
for k in range(len(jobs)):
    interv_m = []
    for j in range(len(jobs[k])):
        interv_m.append(vard[j][k])
    mdl.add(no_overlap(interv_m))

Create a makespan interval variable and link it to some variables using the max global constraint
http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=max#docplex.cp.modeler.max

In [71]:
#on représente le makespan comme une task de durée nulle qu'on va placer à la fin de la dernière task

mymax = mdl.max([mdl.end_of(vard[j][-1]) for j in range(len(jobs))])
makespan = mdl.interval_var(start=(0,upper_bound), end=(0,upper_bound), length=0, name='makespan')
mdl.add(mdl.start_of(makespan) == mymax)

Add now the makespan as an objective function 

In [73]:
mdl.minimize(mdl.end_of(makespan))

Solve this instance. What is the value of the makespan you found. You can print the solution in a format that is easy 
to see visually.  

In [74]:
mdl.solve() #!!!!PB on doit trouver 55

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 44 variables, 38 constraints
 ! Workers              = 1
 ! Presolve             = Off
 ! SearchType           = DepthFirst
 ! Initial process time : 0.04s (0.03s extraction + 0.00s propagation)
 !  . Log search space  : 199.4 (before), 199.4 (after)
 !  . Memory usage      : 372.8 kB (before), 372.8 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         44                 -
 + New bound is 0
 *            75       36  0.05s               (gap is 100.0%)
 *            71       66  0.05s               (gap is 100.0%)
 *            70      132  0.05s               (gap is 100.0%)
 *            69      192  0.05s               (gap is 100.0%)
 *            68      259  0.05s               (gap is 100.0%)
 *            67     

Factorise your code so that it takes as input the data file and it solves the problem. Try to use few other instances https://github.com/tamy0612/JSPLIB/tree/master/instances
    
    


At this stage you are completly free to play. Try different instances, different configurations of the solver, different branching strategies, restarts, randomisation, etc. 

You may want to present your results as a cactus 🌵 plot : in the x-axis we have the runtime, on the y-axis, we have the number of instances solved. Also, some instances are still open in the literature. Have a look here for an up to date list of bounds http://optimizizer.com/TA.php

What did you learn loday? 